In [23]:
import pandas as pd

path = 'data/Result_27.csv'
path_20220617 = 'data/Result_3_202206017.csv'

dfs = pd.read_csv(path_20220617, sep=',', names=['zuber_id', 'intent_id', 'intent_name', 'corpus_id', 'text', 'synonyms'])
dfs.head()

,zuber_id,intent_id,intent_name,corpus_id,text,synonyms
0,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194496,我们可以清晰的可以清晰的可以看到，然后一周呢，能够快速的缓解，缓缓解患者的这个背痛，然后四周...,NaN
1,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194497,可申请可以快速控制炎症，缓解中毒症状，一周缓解背痛四周，缓解晨僵和疲劳，那么快速的缓解晨僵、...,NaN
2,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194498,可善挺可以快速的空盐，缓解中轴的症状呢我们来看一下可善挺的数据，那客栈挺一周就可以缓解背痛四...,NaN
3,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194499,可善挺治疗后一年缓解背痛，四周改善陈江和疲劳在一周内能够快速的缓解他的悲痛，在一个月内的时间...,NaN
4,1000000001,815860776309075968,6IB/推进-FAB-FA-AS,854654812441194500,可身体快速控源，缓解中轴症状，一周即可以缓解背痛四周，缓解晨僵和疲劳。那么咳嗽呢？通过研究结...,NaN


In [24]:
theme_binary = ["中行", "费卡", "6IB", "催收", '赛诺菲', '来优时', '推进']

def get_category(intent_name, theme):
    for t in theme:
        if t in intent_name:
            return t

dfs = dfs.dropna(subset=['text'])
dfs['category'] = dfs['intent_name'].apply(lambda x: get_category(x, theme_binary))

dfs_binary = dfs.dropna(subset=['category'])

dfs_binary = dfs_binary.dropna(subset=['text'])

In [25]:
## 查看intent_name 的总数 -》 考虑转换成标签后存在的总数
from sklearn import preprocessing

names = dfs_binary['intent_name'].drop_duplicates().tolist()
print(f'names length: {len(names)}')


dfs_temp1= dfs_binary[['intent_id', 'intent_name']]
dfs_temp2 = dfs_binary[['intent_id', 'text']]
dfs_temp3 = dfs_binary[['intent_id', 'category']]

intentid2name = dict(zip(dfs_temp1['intent_id'], dfs_temp1['intent_name']))
intentid2text = dict(zip(dfs_temp2['intent_id'], dfs_temp2['text']))
intentid2category = dict(zip(dfs_temp3['intent_id'], dfs_temp3['category']))
corpusid2text = dict(zip(dfs_binary['corpus_id'], dfs['text']))
intent_ids = dfs_binary['intent_id'].drop_duplicates().tolist()
intentid2corpus_id = {}
for intent_id in intent_ids:
    intentid2corpus_id[intent_id] = dfs_binary[dfs_binary['intent_id'] == intent_id]['corpus_id'].tolist()

names length: 148


In [16]:
import pandas as pd

node_path = 'data/tenant_bot_sale_cb_node.csv'
column_names = ['node', 'intents']

dfs_node = pd.read_csv(node_path, names=column_names)

category_counter = []

intents_tables = dfs_node['intents'].tolist()
intents_table_filter = []
for table in intents_tables:
    table_list = eval(table)
    if len(table_list) > 1:
        status = True
        temp_table = []
        for i in table_list:
            if i in intentid2name:
                category_counter.append(intentid2category[i])
                temp_table.append(i)
        if len(temp_table) > 1:
            intents_table_filter.append(temp_table)
print(len(intents_table_filter))
print(intents_table_filter[:3])

559
[[815860776309075968, 815860777558978560], [815860777999380480, 815860778305564672, 815860778603360256], [815860778603360256, 815860778909544448]]


In [17]:
import random

multi_label_data = []


for intent_table in intents_table_filter:
    sample_number = sum([intentid2corpus_id.__len__() for ii in intent_table]) // (len(intent_table) * 2)
    cur_sample = []
    while len(cur_sample) < sample_number:
        text = ''
        cur_corpus_id = []
        table_label = []
        for ii in intent_table:
            i = random.choice(intentid2corpus_id[ii])
            if len(text) + len(corpusid2text[i]) < 125:
                text += corpusid2text[i]
                table_label.append(intentid2name[ii])
        if len(table_label) > 1:
            cur_intent_name = tuple(table_label)
            cur_sample.append((text, intent_table, cur_intent_name))
    multi_label_data.extend(cur_sample)

print(f'total: {len(multi_label_data)}')

total: 41366


In [18]:
dfs_label_corpus = pd.DataFrame(multi_label_data, columns=['text', 'intent_id', 'intent_name'])

dfs_single_label = dfs_binary[['intent_id', 'intent_name', 'text']]

In [19]:
print(dfs_single_label.head(2))

print("****" * 10)
print(dfs_label_corpus.head(2))

            intent_id       intent_name  \
0  815860776309075968  6IB/推进-FAB-FA-AS   
1  815860776309075968  6IB/推进-FAB-FA-AS   

                                                text  
0  我们可以清晰的可以清晰的可以看到，然后一周呢，能够快速的缓解，缓缓解患者的这个背痛，然后四周...  
1  可申请可以快速控制炎症，缓解中毒症状，一周缓解背痛四周，缓解晨僵和疲劳，那么快速的缓解晨僵、...  
****************************************
                                                text  \
0  可申请呢？嗯，可以快速控盐，缓解中轴症状，一周缓解背痛四周，缓解晨僵疲劳。如果患者能够尽早的...   
1  一周可以快速地缓解背痛四周缓解城乡四周缓解疲劳够快速的控制炎症，缓解中轴的症状，一周缓解胃疼...   

                                  intent_id  \
0  [815860776309075968, 815860777558978560]   
1  [815860776309075968, 815860777558978560]   

                             intent_name  
0  (6IB/推进-FAB-FA-AS, 6IB/推进-FAB-FA-PSO)  
1  (6IB/推进-FAB-FA-AS, 6IB/推进-FAB-FA-PSO)  


In [22]:
dfs_label_merge = pd.concat([dfs_single_label, dfs_label_corpus], ignore_index=False)

dfs_label_merge.to_csv('output_data/data_label_corpus_20220628_v1.csv', sep='\t', index=False)